<a href="https://colab.research.google.com/github/somendrew/Practise-with-Projects/blob/main/AutoGluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -U autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500
train_data = train_data.sample(n = subsample_size, random_state = 0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [5]:
train_data.shape

(500, 15)

In [6]:
train_data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'class'],
      dtype='object')

In [7]:
label = 'class'
print('Summary of class variable:\n',train_data[label].describe())

Summary of class variable:
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


In [9]:
train_data['class'].value_counts()

,count
class,
<=50K,365
>50K,135


Now use AutoGluon to train multiple models:

In [10]:
save_path = 'agModels-predictClass' #specify folder to store trained models
predictor = TabularPredictor(label = label, path = save_path).fit(train_data)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       11.33 GB / 12.67 GB (89.4%)
Disk Space Avail:   65.07 GB / 107.72 GB (60.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with fast inference s

Next, load separate test data to demonstrate how to make predictions on new examples at inference time:

In [11]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [12]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Predictions:  
 0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [13]:
perf

{'accuracy': 0.8409253761899887,
 'balanced_accuracy': np.float64(0.7475663839529563),
 'mcc': np.float64(0.5345297121913682),
 'f1': 0.6296472831267874,
 'precision': 0.7034078807241747,
 'recall': 0.5698878343399483}

In [14]:
results = predictor.fit_summary()
results

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0               XGBoost       0.86    accuracy       0.008183  0.389230                0.008183           0.389230            1       True         11
1   WeightedEnsemble_L2       0.86    accuracy       0.009203  0.502977                0.001020           0.113747            2       True         14
2              LightGBM       0.85    accuracy       0.005021  0.310243                0.005021           0.310243            1       True          4
3              CatBoost       0.85    accuracy       0.011193  2.098637                0.011193           2.098637            1       True          7
4       NeuralNetFastAI       0.84    accuracy       0.028336  2.155315                0.028336           2.155315            1       True         10
5      RandomForestGini       0.84    

{'model_types': {'KNeighborsUnif': 'KNNModel',
  'KNeighborsDist': 'KNNModel',
  'LightGBMXT': 'LGBModel',
  'LightGBM': 'LGBModel',
  'RandomForestGini': 'RFModel',
  'RandomForestEntr': 'RFModel',
  'CatBoost': 'CatBoostModel',
  'ExtraTreesGini': 'XTModel',
  'ExtraTreesEntr': 'XTModel',
  'NeuralNetFastAI': 'NNFastAiTabularModel',
  'XGBoost': 'XGBoostModel',
  'NeuralNetTorch': 'TabularNeuralNetTorchModel',
  'LightGBMLarge': 'LGBModel',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif': 0.73,
  'KNeighborsDist': 0.65,
  'LightGBMXT': 0.83,
  'LightGBM': 0.85,
  'RandomForestGini': 0.84,
  'RandomForestEntr': 0.83,
  'CatBoost': 0.85,
  'ExtraTreesGini': 0.82,
  'ExtraTreesEntr': 0.81,
  'NeuralNetFastAI': 0.84,
  'XGBoost': 0.86,
  'NeuralNetTorch': 0.83,
  'LightGBMLarge': 0.83,
  'WeightedEnsemble_L2': 0.86},
 'model_best': 'WeightedEnsemble_L2',
 'model_paths': {'KNeighborsUnif': ['KNeighborsUnif'],
  'KNeighborsDist': ['KNeighborsDist'

In [15]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  binary
AutoGluon identified the following types of features:
('category', [])  : 7 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
('int', [])       : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
('int', ['bool']) : 1 | ['sex']


In [16]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842870,0.84,accuracy,0.660251,0.079817,0.957869,0.660251,0.079817,0.957869,1,True,5
1,CatBoost,0.842461,0.85,accuracy,0.021798,0.011193,2.098637,0.021798,0.011193,2.098637,1,True,7
2,RandomForestEntr,0.841130,0.83,accuracy,0.388485,0.078485,0.736171,0.388485,0.078485,0.736171,1,True,6
3,XGBoost,0.840925,0.86,accuracy,0.136364,0.008183,0.389230,0.136364,0.008183,0.389230,1,True,11
4,WeightedEnsemble_L2,0.840925,0.86,accuracy,0.139062,0.009203,0.502977,0.002698,0.001020,0.113747,2,True,14
5,LightGBM,0.839799,0.85,accuracy,0.177150,0.005021,0.310243,0.177150,0.005021,0.310243,1,True,4
6,LightGBMXT,0.836421,0.83,accuracy,0.039203,0.004547,5.768017,0.039203,0.004547,5.768017,1,True,3
7,ExtraTreesGini,0.833862,0.82,accuracy,0.377765,0.090182,0.767138,0.377765,0.090182,0.767138,1,True,8
8,ExtraTreesEntr,0.833862,0.81,accuracy,0.390039,0.090105,0.764382,0.390039,0.090105,0.764382,1,True,9
9,NeuralNetTorch,0.833657,0.83,accuracy,0.135077,0.012316,8.693242,0.135077,0.012316,8.693242,1,True,12


In [17]:
predictor.predict(test_data, model='LightGBM')

,class
0,<=50K
1,<=50K
2,>50K
3,<=50K
4,<=50K
...,...
9764,<=50K
9765,<=50K
9766,<=50K
9767,<=50K


Maximizing predictive performance

In [18]:
time_limit = 60  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')
predictor.leaderboard(test_data, silent=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20250513_053858"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.94 GB / 12.67 GB (86.3%)
Disk Space Avail:   65.02 GB / 107.72 GB (60.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,0.900085,0.891223,roc_auc,1.597265,0.090750,33.325523,1.597265,0.090750,33.325523,1,True,3
1,WeightedEnsemble_L3,0.900085,0.891223,roc_auc,1.599512,0.091791,33.361820,0.002247,0.001042,0.036297,3,True,5
2,WeightedEnsemble_L2,0.900085,0.891223,roc_auc,1.600371,0.091719,33.355290,0.003105,0.000969,0.029766,2,True,4
3,KNeighborsDist_BAG_L1,0.525998,0.536956,roc_auc,0.052676,0.020106,0.006362,0.052676,0.020106,0.006362,1,True,2
4,KNeighborsUnif_BAG_L1,0.514970,0.519604,roc_auc,0.040510,0.034220,0.012815,0.040510,0.034220,0.012815,1,True,1
